<a href="https://colab.research.google.com/github/Songhyunjeong/thingplug-lora-starter-kit/blob/master/notebookd5c4df25c8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'postpia4:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4116903%2F7134966%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T075404Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8b56223f6170f818e9a6176fa9e63cedf4b25a6f2a1b374ed0e40b76dba7b4d5e5f299ffe0ddf6c63b867f8a0c6b1c08eb169b3b610b478aff7dad09d13242b25c68e1e4581a3aabac5bfc1f5a50261e99538d785b3c1eefc29f017d22272456c3a9ce3684d9de91bfe73c4b85493031f73b5f0794e9158cb3cfee05f81e073d22efa8d7bf94bad18918d0fc7aa0e0dc72d36d4a657cd092976aed3765934ae2a098eb2dea24b4c51bedccb4ac2ad74a9a89d07194ff063b97ed9cb70a5388dbf83731e4900e03e0b995fe18ffcae79a3c47e71f96dfcd528445721b2a5da9ff0fcd01075e8ccc3c8a720ae5b00719a67f9a9e63210898e654ec84d10d035738,basic-01:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4483020%2F7683464%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T075404Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D80f731e43fa1c2ff224c95802b5d4daca786ff45f8653a0c3dbd3d1a3ebba52015ba968c99e34cddf4f9cff4768bb6a514e15a71053fdecde69e8b41ec7c166f79173d26931eee286b765d796a9164fbb525da14e44549fdfca6032c90f0919e0af27a04b4c6a6839b334797707c0e8136872b7b59115d40985c3daf09a1deb55c1c587b506de517721ce147123c30e6127d7170efa9739f0deae4ea11b82873792f760a303e42a270af5ff5be925723cdfd3c9240cdeb301a0f78b6ce131f158f8413ebd7a8549e24d2b6d31bbf1553ef9d3eade9fc8016b083057e764b9103357d8fd054aa7c496d4226bf50d6f2334ce65d2be047b82ef720981c956e4c4b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/postpia4/opendata.pdf
/kaggle/input/basic-01/basic_manual.pdf


아래 코드를 차례대로 실행하거나 Run All 실행하세요.원하는 파일(PDF)을 업로드하여 변경 적용해주시면 됩니다.샘플로 보시고 업무에 적용해 보시면 좋을 듯합니다.(카피해서 새로운 노트북 생성해서 사용하셔도 됩니다)
키값은 유료계정으로 발급받은 키값이고 현재 약 20달러 충전되어 있습니다. 많이 사용해도 얼마 차감안되니 마음껏 사용하셔도 됩니다.

In [ ]:
!pip install -U langchain-openai

!pip install openai
!pip install chromadb
!pip install tiktoken

  Obtaining dependency information for langchain-openai from https://files.pythonhosted.org/packages/b9/66/e96a5b70b453c4f53830abee34208a18bdb8ad0c1765cbfbf80938d8b975/langchain_openai-0.0.7-py3-none-any.whl.metadata
  Obtaining dependency information for openai<2.0.0,>=1.10.0 from https://files.pythonhosted.org/packages/26/a1/75474477af2a1dae3a25f80b72bbaf20e8296191ece7fff2f67984206f33/openai-1.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for tiktoken<1,>=0.5.2 from https://files.pythonhosted.org/packages/16/05/5efbd91252ffb1301ea393d88ef736b33d41e75d4bcf0bd31d660050e400/tiktoken-0.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/41/7b/dda

In [21]:
from google.colab import userdata
userdata.get('my_openai_key')

from dotenv import load_dotenv
load_dotenv()

# PyPDFLoader
from langchain.document_loaders import PyPDFLoader

# TextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Loader
loader = PyPDFLoader("/kaggle/input/basic-01/basic_manual.pdf")
pages = loader.load_and_split()




# Split
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 300,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)
texts = text_splitter.split_documents(pages)
print(texts[0])

# Embedding
from langchain_openai import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings(openai_api_key="my_openai_key")

# load it into Chroma
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, embeddings_model)

# Question & Answer
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0,openai_api_key="my_openai_key")
qa_chain = RetrievalQA.from_chain_type(llm,retriever=db.as_retriever())




page_content='공공데이터베이스 \n표준화 관리 \n매뉴얼 \n2023. 4.\n목 차\nⅠ. 공공데이터베이스 표준화 총론\n1. 공공데이터베이스 표준화 개요· · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·  2' metadata={'source': '/kaggle/input/basic-01/basic_manual.pdf', 'page': 0}


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: my_opena*_key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
!pip install pypdf
!pip install langchain

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/45/05/2d5c38c7fa1a091a61e018070b6878410d256be577e5154843d339e59b2c/langchain-0.1.9-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpatch<2.0,>=1.33 from https://files.pythonhosted.org/packages/73/07/02e16ed01e04a374e644b575638ec7987ae846d25ad97bcc9945a3ee4b0e/jsonpatch-1.33-py2.py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community<0.1,>=0.0.21 from https://files.pythonhosted.org/packages/ae/98/3066c36e74f916ccb875afa40d9a4cee28755335ce7b6c675c25d4572920/langchain_community-0.0.22-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.2,>=0.1.26 from https://files.pythonhosted.org/packages/27/a0/6cd072db5ff0dec6b080575c0d976fc1bf22063d131f038496c650db7b84/langchain_core-0.1.26-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.2.0,>=0.1.0 from https://files.pythonhosted.org/packages/91/ff/a3

In [ ]:

my_dict = {}


while True:
    # 사용자로부터 질문 입력 받기
    question = input("질문을 입력하세요 (종료하려면 '종료'를 입력하세요): ")

    # '종료' 입력 시 루프 종료
    if question == '종료':
        break

    # 답변 생성 (여기서는 간단히 예시로 "질문에 대한 답변"으로 대체)
    answer = qa_chain({"query": question})

    # 딕셔너리에 질문과 답변 추가
    my_dict[question] = answer
    print(answer)


with open("질문과_답변.txt", "w", encoding="utf-8") as file:
    for question, answer in my_dict.items():
        file.write(f"질문: {question}\n")
        file.write(f"답변: {answer}\n\n")

print("질문과 답변이 파일로 저장되었습니다.")

질문을 입력하세요 (종료하려면 '종료'를 입력하세요):  공공데이터베이스 산출물의 종류는?


NameError: name 'qa_chain' is not defined

In [ ]:
!pip install openai

In [ ]:
python --version


NameError: name 'python' is not defined